In [46]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [47]:
df = pd.read_csv('train.csv')

In [48]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [49]:
from tensorflow.keras.layers import TextVectorization

In [50]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [51]:
MAX_FEATURES = 200000 # number of words in the vocab
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)
vectorized_text.shape

TensorShape([159571, 1800])

In [52]:
import tensorflow as tf
import gradio as gr
import numpy as np

In [53]:
model = tf.keras.models.load_model('toxicity.h5')

In [54]:
input_str = vectorizer('hey i freaken hate you!')

In [55]:
res = model.predict(np.array([input_str]))

1/1 [==============================] - 2s 2s/step


In [56]:
res

array([[0.68677735, 0.02167605, 0.31519976, 0.02943351, 0.3220586 ,
        0.06581379]], dtype=float32)

In [57]:
(res > 0.5).astype(int)

array([[1, 0, 0, 0, 0, 0]])

In [58]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [59]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [60]:
interface = gr.Interface(fn=score_comment, 
                         inputs="text",
                        outputs='text')
interface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 249ms/step
